In [ ]:
import os
import iptcinfo3
import time
import xlrd
import shutil
from PIL import Image
import tkinter as tk
from tkinter.filedialog import askdirectory,askopenfilename
from pathlib import Path
from urllib.request import quote
import logging
 
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s | %(funcName)s - 第 %(lineno)d 行 - %(message)s')
logger = logging.getLogger(__name__)

#     logger.info('This is a log info')
#     logger.debug('Debugging')
#     logger.warning('Warning exists')

class dzxc_tool(): 
    def __init__(self):
        root=tk.Tk()
        root.title('大智小超')
        root.geometry('600x300')
        root.resizable(0,0)

        fm1=tk.Frame(root)
        fm2=tk.Frame(root)
        fm3=tk.Frame(root)


        for fm in (fm1,fm2,fm3):
            fm.grid(row=0,column=0, sticky='news')

        deal_pic_wtmk_rename(fm1)
        pic_to_stu(fm2)
        to_html_stu(fm3)

        menubar=tk.Menu(root)
        menubar.add_command(label='加水印及改名',command=lambda:self.raise_frame(fm1))
        menubar.add_command(label='复制照片到cos文件夹',command=lambda:self.raise_frame(fm2))
        menubar.add_command(label='生成学员html',command=lambda:self.raise_frame(fm3))

        self.raise_frame(fm1)
        root.config(menu=menubar)
        root.mainloop()
        
    def raise_frame(self,frame):  
        frame.tkraise() 
    
class deal_pic_wtmk_rename():
    def __init__(self,fr):
        self.fr=fr
        self.pic_wtmk_rename(self.fr)
    
    def pic_wtmk_rename(self,dzxc):

        
        #标题       
        self.title = tk.Label(dzxc,bg='grey',width=55,height=2,text='图片改名、加水印及分辨率处理',\
                     font=('microsoft yahei', 13, 'bold'),fg='white',anchor='center')
        self.title.grid(row=0,column=0,sticky='w')
        
        #logo
#         self.logo=tk.PhotoImage(file='I:\\py\\logo.png')
#         self.img_logo=tk.Label(dzxc,image=self.logo,width=50,height=48)
#         self.img_logo.grid(row=0,column=0,sticky='w')      
        


        # 选择要处理的目录
        _dir=tk.StringVar()
        _dir_disp=tk.StringVar()
        _btn2=tk.Button(dzxc,text='请选择要处理的文件夹',width=30,height=1,command=lambda:self.select_path(_dir,_dir_disp))
        _btn2.grid(row=1,column=0,pady=5)

        deal_msg=tk.StringVar()
        _btn3=tk.Button(dzxc,text='处 理',width=30,height=1,command=lambda:self.wtmk(_dir.get(),deal_msg))
        _btn3.grid(row=2,column=0,pady=5)   

        #显示要处理的目录
        t2=tk.Label(dzxc,bg='#fffeef',width=86,height=2,fg='#5f5f5f',textvariable=_dir_disp,anchor='center')
        t2.grid(row=3,column=0,sticky='w')

        #显示处理后的信息    
        t3=tk.Label(dzxc,bg='#ffffff',width=61,height=6,\
                    fg='#0325a0',font=('microsoft yahei', 12),textvariable=deal_msg,anchor='center')
        t3.grid(row=4,column=0,sticky='w')   
      
    
    def select_path(self,_dir,_dir_disp):
        p=askdirectory()
        _dir_disp.set('要处理的文件夹为：'+p)
        _dir.set(Path(p))    

    def wtmk(self,_dir,deal_msg):    
    #     _dir=input('输入图片所在文件夹名称：')
    #     path=os.path.join('I:\\每周实验课',_dir)
        self.path=Path(_dir)
        self.course_name=str(self.path).split("\\")[-1][5:]     
        

        for root,dirs,files in os.walk(self.path):
            k=0
            for file in files:
                if file[-3:].lower()=='jpg':
                    try:                       
                        pic_name=os.path.join(self.path,file)  
                        logger.debug(pic_name)
                        self.tag_before=iptcinfo3.IPTCInfo(pic_name)['keywords'] #读取JPG文件标记 
                        logger.debug(self.tag_before)
                        print(code_to_str(self.tag_before))
                        wtmk_name=self._wtmk(self.course_name) #水印的路径+名称
                        pic_after_wtmk=self.put_mark(pic_name,wtmk_name)
#                         pic_after_wtmk.save(pic_name)
                        self.tag_after=iptcinfo3.IPTCInfo(pic_name)
#                         self.tag_after['keyword']=self.tag_before#重新写入JPG文件标记  
#                         self.tag_after.save()                  
                        k+=1      
                    except:
                        pass
        _pics_disp=self.rename_weektotal(str(self.path))

        s=''
        for _key in _pics_disp.keys():
            s=s+_key+' 文件'+str(_pics_disp[_key])+' 个，'
        s=s[:-1]    

        msg='文件夹中包含有{0}。\n\n 已为{1}个JPG文件添加了水印并改名，分辨率为1200×900。'.format(s,k)
        deal_msg.set(msg)

        logger.debug('...all done...')

    def _wtmk(self,_name):
        path='I:\\每周课程标签\\'
        name=path+'水印-'+_name+".png"
        return name

    def put_mark(self,pic_addr,wtmk_addr):
        _pic=Image.open(pic_addr)
        wtmk_size=(310,int(310*176/756))
        wtmk=Image.open(wtmk_addr).convert("RGBA").resize(wtmk_size)    
        pic=_pic.resize((1200,900))
        pic.paste(wtmk,(48,785),mask=wtmk)    
        return pic

    def rename_weektotal(self,_path):
        types=[]    
        y=time.strftime('%Y'.format(time.localtime()))
        m=_path.split('\\')[-1][:2]
        d=_path.split('\\')[-1][2:4]
        course_name=_path.split('\\')[-1][5:]
        pic_date=[y,m,d]


        for root,dirs,files in os.walk(_path):
            for file in files:
                if not file[-3:] in types:
                    if file[-3:].lower()=='jpg' or file[-3:].lower()=='jpeg' or file[-3:].lower()=='mov' or file[-3:].lower=='mp4':
                        types.append(file[-3:].upper())

        msg=[]
        for type in types:
            _msg=[]

            k=1
            for root,dirs,files in os.walk(_path):
                for file in files:                
                    if file[-3:]==type:
                        o_name=_path+'\\'+file
                        n_name=_path+'\\'+'-'.join(pic_date)+"-"+course_name+"-"+str(k).zfill(3)+"."+type
                        logger.debug(o_name+' ----> '+n_name)
#                         os.rename(o_name,n_name)
                        k+=1
            _msg=[type,k-1]
            msg.append(_msg)

        return dict(msg)        
        
class pic_to_stu(): 
    def __init__(self,fr):
        self.fr=fr
#         self.to_each_dir(self.fr)
        #标题
        self.title = tk.Label(self.fr,bg='grey',width=55,height=2,text='学员文件夹照片重命名，将照片复制到cos文件夹。',\
                     font=('microsoft yahei', 13, 'bold'),fg='white',anchor='center')
        self.title.grid(row=0,column=0,columnspan=2)
        #logo
#         self.logo=tk.PhotoImage(file='I:\\py\\logo.png')
#         self.img_logo=tk.Label(self.fr,image=self.logo,width=50,height=48)
#         self.img_logo.grid(row=0,column=0,sticky='w')
            
        self.month_msg=tk.Label(self.fr,width=10,text='请输入月份：')
        self.month_msg.grid(row=1,column=0,sticky='e',pady=10)
        self.month_entry=tk.Entry(self.fr,show=None,width=10)
        self.month_entry.grid(row=1,column=1,sticky='w',pady=10)
        
        self.btn=tk.Button(self.fr,width=30,text='处 理',command=lambda:self.renname_dis_stu(self.month_entry.get()))
        self.btn.grid(row=2,column=0,pady=10,columnspan=2)
        
    def renname_dis_stu(self,month):
        self.month=month
        self.rename_wx_pic(self.month)
        self.to_cos_dir(self.month)
    
    
    def to_cos_dir(self,month):
        logger.debug('处理分配的程序')
        self.month=month
        self.path='I:\\每周实验课_学员'
        self.year='2019'
        self.des_path=os.path.join(self.path,'cos',self.year,str(self.month).zfill(2)+'月')
        logger.debug(mkdir(self.des_path))
        for dir in os.listdir(self.path):
            self.st_dir_name=os.path.join(self.path,dir)
            if os.path.isdir( self.st_dir_name):
                if os.path.split( self.st_dir_name)[-1]!='html' and os.path.split( self.st_dir_name)[-1]!='cos':                   
                    for file in os.listdir( self.st_dir_name):
                        month_in_file=file[file.find('-')+1:file.find('-')+3]
                        if month_in_file==self.month:
                            self.src_file=os.path.join( self.st_dir_name,file)
                            self.des_file=os.path.join(self.des_path,file)
                            logger.debug(self.src_file+'  --->  '+self.des_file)
                            shutil.copyfile(self.src_file,self.des_file)

        
    def rename_wx_pic(self,month):
#         month=input("月份：")
        path='I:\\每周实验课_学员'
        for dir in os.listdir(path):
            st_dir_name=os.path.join(path,dir)
            if os.path.isdir(st_dir_name):
                if os.path.split(st_dir_name)[-1]!='html' and os.path.split(st_dir_name)[-1]!='cos':                   
                    logger.debug(st_dir_name)
                    k=0
                    for _files in os.listdir(st_dir_name):
                        if _files[-3:]=="jpg" or _files[-3:]=="JPG":
                            if _files[5:7]==month and  len(_files)>14:
                                oldname=os.path.join(st_dir_name,_files)
                                newname=st_dir_name+"\\"+dir+_files[4:8]+str((k+1)).zfill(3)+".jpg"
                                os.rename(oldname,newname)
                                print(oldname, "===>>",newname)
                                k+=1
                    if k==0:
                        print("{0}月，{1} 没有要修改的文件名。\n".format(month,dir))
        
class to_html_stu():  
    def __init__(self,fr):
        self.fr=fr
        #标题
        self.title = tk.Label(self.fr,bg='grey',width=55,height=2,text='生成学员公众号html代码',\
                     font=('microsoft yahei', 13, 'bold'),fg='white',anchor='center')
        self.title.grid(row=0,column=0,columnspan=2,sticky='w')
        #logo
#         self.logo=tk.PhotoImage(file='I:\\py\\logo.png')
#         self.img_logo=tk.Label(self.fr,image=self.logo,width=50,height=48)
#         self.img_logo.grid(row=0,column=0,sticky='w')
        
        month_pre=tk.Label(self.fr,show=None, width=13,text='请输入月份 ')
        month=tk.Entry(self.fr, width=10)
        month_pre.grid(row=1,column=0,pady=10,sticky='e')
        month.grid(row=1,column=1,sticky='w')
        
        msg=tk.StringVar()
        msgbox=tk.Label(self.fr,textvariable=msg,width=90,height=18,bg='white',anchor='n')
        msgbox.grid(row=3,column=0,sticky='N',columnspan=2)        
        
        btn=tk.Button(self.fr,text='处 理',width=20,command=lambda:self.do_html(self.fr,month.get(),msg))
        btn.grid(row=2,column=0,columnspan=2,pady=20)          
                     
    
    def do_html(self,dzxc,month,msg):
        logger.debug(month+'月')
    #     month=input('输入月份：')
        dirs=self.walkDir()
        _msgs=[]
        for dir in dirs:
            txt=self.to_html(dir,month)
            _msg=self.to_file(txt)
            if _msg:
                _msgs.append(_msg)
        self.to_msgbox(msg,'\n'.join(_msgs))
    
    def to_html(self,thename,month):
        mtoM={"1":"一月","2":"二月","3":"三月","4":"四月","5":"五月","6":"六月","7":"七月","8":"八月","9":"九月","10":"十月","11":"十一月","12":"十二月"}
        # month=input('月份：')
        # month='7'
        filepath='I:\\每周实验课_学员\\'
        filename=filepath+'2019科学实验课学员档案.xlsx'
        data = xlrd.open_workbook(filename)
        table_total = data.sheet_by_name('学员名单')
        course_total = table_total.row_values(2, 12)
        del course_total[-1]

        names = table_total.col_values(2, 3)  # 第3列，从第4行开始
        inf_basic = []
        course = []
        for n in names:
            if n == thename:
                tick_row = names.index(thename) + 3

                for j in range(12, table_total.ncols - 1):
                    if table_total.cell(tick_row, j).value == '√':
                        course.append([self.xlMonth(table_total.cell(1, j).value), table_total.cell(2, j).value])

        title_course=''
        course_to_comment=[]
        for i in course:
            if i[0]==month:
                title_course=title_course+i[1]+' | '
                course_to_comment.append(i[1])

        logger.debug(thename,title_course)
        #判断该学员是否在该月有课程，如无，后面则不生成文件。
        if title_course:
            out_yn=1
        else:
            out_yn=0

        html_title=[mtoM[month],title_course[:-3]]

        t1 = '''<p style="color: #1e9be8;line-height: 2em;">
                                    <strong style="color: inherit;" class="135brush" data-brushtype="text">{0}月&nbsp;</strong><span style="caret-color: red;font-size: 12px;">{1}</span>
                                </p>
            '''.format(month,html_title[1])

        #处理图片地址
    # 旧图片地:（按人名建文件夹）：
    #     photo_path='I:\\每周实验课_学员\\'
    #     dir_name=photo_path+thename

    #新图片地址（按月份，不按人名建文件夹）：
        photo_path='I:\\每周实验课_学员\\cos\\2019\\'
        dir_name=photo_path+str(month).zfill(2)+"月\\"
        html_pic_addr=''
        _pre_pic='''
        </section>
                        <section style="color: #bfbfbf;padding-top: 10px;padding-bottom: 10px;display: inline-block;width: 100%;box-sizing: border-box;" data-width="75%">
                            <section class="_135editor" data-tools="135编辑器" data-id="95138">
                                <section class="_135editor">
                                    <section style="margin: 1em auto;text-align: center;padding: 5px;border-width: 1px;border-style: solid;border-color: transparent;overflow: hidden;box-sizing: border-box;">
                                        <section class="135brush" data-style="display: inline-block;width: 100%;margin:0;padding:0;" style="white-space: nowrap;overflow-x: scroll;">
        '''

#336行地址有’2019‘，到2020年时应该改为’2020‘
        pre_pic_addr='''
         <img class="" data-ratio="0.75" data-type="jpeg" data-w="1200" data-width="100%" src="https://chuntianhuahua-1257410889.cos.ap-guangzhou.myqcloud.com/dzxc/2019/
         '''
        after_pic_addr='"/>'
        for root,dirs,files in os.walk(dir_name):
            for file in files:
    #             if len(file) > 5 and file[-3:] == 'jpg' :# 旧的判断
                if file[0:-8]==thename+"-"+str(month).zfill(2):# 旧的判断
    #                 if file[5:7]==month.zfill(2):#,旧的if                    
    #                     a='{0}{1}/{2}{3}'.format(pre_pic_addr.rstrip(),quote(thename),file,after_pic_addr) #旧的文件名
                    a='{0}{1}/{2}{3}'.format(pre_pic_addr.rstrip(),quote(str(month).zfill(2)+"月"),quote(file),after_pic_addr)
    #                 logger.debug(a)
                    html_pic_addr=html_pic_addr+a
        html_pic_addr=_pre_pic+html_pic_addr
    #     logger.debug(html_pic_addr)


        #处理课程评论
        table_exp = data.sheet_by_name('实验课内容')
        comments = table_exp.col_values(1, 1)
        cmt=''
        html_cmt=[]
        for c in course_to_comment:
            for i in comments:
                if i==c:
                    logger.debug(comments.index(c)+1)
                    _cmt=table_exp.cell(comments.index(c)+1,4).value
                    cmt=cmt+_cmt+'\n'
        html_cmt=cmt.split('\n')

        t2='''
                                    <p style="text-align:center; box-sizing: border-box; margin-right: 0px; margin-bottom: 0px; margin-left: 0px; padding: 0px; font-weight: 300; color: #262626; overflow-wrap: break-word; font-size: 14px; margin-top: 0px !important; font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Roboto, &quot;Helvetica Neue&quot;, &quot;PingFang SC&quot;, &quot;Noto Sans&quot;, &quot;Noto Sans CJK SC&quot;, &quot;Microsoft YaHei&quot;, 微软雅黑, sans-serif;" align="center">
                                        <span style="font-size: 12px;color: #7f7f7f;">{0}在科学实验室里，<br/></span>
                                    </p>
        '''.format(mtoM[month])

        t3=''
        for html in html_cmt:
            if html:
                _t3='''
                                    <p style="text-align:center;box-sizing: border-box; margin: 0px; padding: 0px; font-weight: 300; color: #262626; overflow-wrap: break-word; font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Roboto, &quot;Helvetica Neue&quot;, &quot;PingFang SC&quot;, &quot;Noto Sans&quot;, &quot;Noto Sans CJK SC&quot;, &quot;Microsoft YaHei&quot;, 微软雅黑, sans-serif;" align="center">
                                        <span style="font-size: 12px;color: #7f7f7f;">{0}</span>
                                    </p>
                '''.format(html)
                t3=t3+_t3

        pre_t='''
        <section data-role="outer" label="Powered by 135editor.com" style="font-size:16px;">
        <section class="_135editor" data-tools="135编辑器" data-id="86457" data-color="#1e9be8" style="font-size: 16px;">
            <section class="_135editor" style="font-size: 16px;border-width: 0px;border-style: none;border-color: initial;box-sizing: border-box;" data-color="#1e9be8">
                <section style="width: 1.2em;height: 1.2em;border-radius: 100%;background-color: #1e9be8;display: inline-block;margin-top: 1em;vertical-align: top;color: #ffffff;box-sizing: border-box;"></section>
                <section style="display: inline-block;width: 95%;border-left: 1px solid #1e9be8;border-top-color: #1e9be8;border-right-color: #1e9be8;border-bottom-color: #1e9be8;margin-left: -0.66em;box-sizing: border-box;" data-width="95%">
                    <section style="margin-top: 2px;color: inherit;display: inline-block;width: 100%;padding-left: 15px;box-sizing: border-box;" data-width="100%">
                        <section style="color: #bfbfbf;padding: 5px;display: inline-block;width: 100%;float: left;box-sizing: border-box;" data-width="100%">
        '''

        after_t='''
        </section>
                                        <p style="line-height:32px;margin: 10px;">
                                            <span style="font-size: 12px;">左右滑动查看更多</span>
                                        </p>
                                    </section>
                                </section>
                            </section>
                        </section>
                    </section>
                </section>
            </section>
        </section>
        '''
        html_text=pre_t+t1+t2+t3+html_pic_addr+after_t

        out=[thename,month,html_text,out_yn]
#         logger.debug(out)
        return out

    def to_file(self,txt):
        # path=os.getcwd()+'\\' #要改为每个学员的文件夹
#         logger.debug(txt)        
        if txt[3]==1:
            path = 'I:\\每周实验课_学员\\html\\'
            filename=path+txt[1].zfill(2)+'-'+self.single_get_first(txt[0][0:1]).upper()+"-"+txt[0]+'.html'
            f=open(filename,'w+')
            f.writelines(txt[2])
            self.out_msg='生成文件：{0}'.format(filename)          
            f.close()
        else:
            if txt[0]!='html' and txt[0]!='cos' and txt[0]!='z要洗的照片':
                self.out_msg='{0} 在 {1} 月未上课。'.format(txt[0],txt[1].zfill(2))
            else:
                self.out_msg=''
        logger.debug(self.out_msg)     
        return self.out_msg
                
    def to_msgbox(self,msg,txt):
        logger.debug(txt)
        msg.set(txt)

    def walkDir(self):
        path='I:\\每周实验课_学员'
        out=[]
        for dir in os.listdir(path):
            dir=os.path.join(path,dir)
            if os.path.isdir(dir):
                out.append(os.path.split(dir)[-1])
        return out

    def xlDate(self,date):
        d=xlrd.xldate_as_tuple(date,0)
        #dd=str(d[0])+'-'+str(d[1])+'-'+str(d[2])
        dd = str(d[1]) + '月' + str(d[2])+ '日'
        return dd

    def xlMonth(self,date):
        d=xlrd.xldate_as_tuple(date,0)
        #dd=str(d[0])+'-'+str(d[1])+'-'+str(d[2])
        dd = str(d[1])
        return dd

    def selectPath(self):
        path_=askdirectory()
        # path.set(path_)
        logger.debug(path_)
        path0= StringVar()
        path0.set(path_)
        Label(root, textvariable=path0).grid(row=1, column=0)
        rename(path_)
        
    def single_get_first(self,unicode1):
        str1 = unicode1.encode('gbk')
        logger.debug(len(str1))
        try:
            ord(str1)
            return str1
        except:
            asc = str1[0] * 256 + str1[1] - 65536
            logger.debug(asc)
            if asc >= -20319 and asc <= -20284:
                return 'a'
            if asc >= -20283 and asc <= -19776:
                return 'b'
            if asc >= -19775 and asc <= -19219:
                return 'c'
            if asc >= -19218 and asc <= -18711:
                return 'd'
            if asc >= -18710 and asc <= -18527:
                return 'e'
            if asc >= -18526 and asc <= -18240:
                return 'f'
            if asc >= -18239 and asc <= -17923:
                return 'g'
            if asc >= -17922 and asc <= -17418:
                return 'h'
            if asc >= -17417 and asc <= -16475:
                return 'j'
            if asc >= -16474 and asc <= -16213:
                return 'k'
            if asc >= -16212 and asc <= -15641:
                return 'l'
            if asc >= -15640 and asc <= -15166:
                return 'm'
            if asc >= -15165 and asc <= -14923:
                return 'n'
            if asc >= -14922 and asc <= -14915:
                return 'o'
            if asc >= -14914 and asc <= -14631:
                return 'p'
            if asc >= -14630 and asc <= -14150:
                return 'q'
            if asc >= -14149 and asc <= -14091:
                return 'r'
            if asc >= -14090 and asc <= -13119:
                return 's'
            if asc >= -13118 and asc <= -12839:
                return 't'
            if asc >= -12838 and asc <= -12557:
                return 'w'
            if asc >= -12556 and asc <= -11848:
                return 'x'
            if asc >= -11847 and asc <= -11056:
                return 'y'
            if asc >= -11055 and asc <= -10247:
                return 'z'
            return ''
        
def mkdir(path): 
    folder = os.path.exists(path)
    if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path) 
        return '建立了新文件夹'
    else:
        return '文件夹已存在'
    
def code_to_str(s):
    s=s['keywords']
    if isinstance(s,list):
        out=''
        for i in s:
            out=out+i.decode('utf-8')+'\n'
    else:
        out=s.decode('utf-8')
   
    print(out)
    return out
                    
if __name__=='__main__':
    dzxc_tool()


DEBUG | wtmk - 第 110 行 - I:\每周实验课\1221-电路萤火虫\img-00001.JPG
INFO | scanToFirstIMMTag - 第 716 行 - File is JPEG, proceeding with JpegScan
DEBUG | jpeg_next_marker - 第 208 行 - jpeg_next_marker: at marker E0 (224)
DEBUG | jpeg_get_variable_length - 第 176 行 - JPEG variable length: 16
DEBUG | jpeg_next_marker - 第 208 行 - jpeg_next_marker: at marker E1 (225)
DEBUG | jpeg_get_variable_length - 第 176 行 - JPEG variable length: 4204
DEBUG | jpeg_next_marker - 第 208 行 - jpeg_next_marker: at marker E1 (225)
DEBUG | jpeg_get_variable_length - 第 176 行 - JPEG variable length: 3098
DEBUG | jpeg_next_marker - 第 208 行 - jpeg_next_marker: at marker DB (219)
DEBUG | jpeg_get_variable_length - 第 176 行 - JPEG variable length: 67
DEBUG | jpeg_next_marker - 第 208 行 - jpeg_next_marker: at marker DB (219)
DEBUG | jpeg_get_variable_length - 第 176 行 - JPEG variable length: 67
DEBUG | jpeg_next_marker - 第 208 行 - jpeg_next_marker: at marker C0 (192)
DEBUG | jpeg_get_variable_length - 第 176 行 - JPEG variable length: 